**Download and Preprocess the Dataset**

In [ ]:
!kaggle datasets download stanfordu/stanford-question-answering-dataset
!unzip /content/stanford-question-answering-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/stanfordu/stanford-question-answering-dataset
License(s): CC-BY-SA-4.0
 80% 7.00M/8.73M [00:01<00:00, 8.30MB/s]
100% 8.73M/8.73M [00:01<00:00, 6.03MB/s]
Archive:  /content/stanford-question-answering-dataset.zip
  inflating: dev-v1.1.json           
  inflating: train-v1.1.json         


In [ ]:
!unzip /content/stanford-question-answering-dataset.zip

Archive:  /content/stanford-question-answering-dataset.zip
replace dev-v1.1.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train-v1.1.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import json
from collections import defaultdict

def create_formatted_json(file_path, output_path):
    with open(file_path, 'r') as file:
        squad_data = json.load(file)

    formatted_data = []

    for entry in squad_data['data']:
        for paragraph in entry['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                answer = qa['answers'][0]['text'] if qa['answers'] else ""

                formatted_entry = {
                    "input": f"Context: {context}\nQuestion: {question}\nAnswer:",
                    "label": answer
                }
                formatted_data.append(formatted_entry)

    with open(output_path, 'w') as output_file:
        json.dump(formatted_data, output_file, indent=4)

In [ ]:
create_formatted_json('/content/train-v1.1.json', 'squad_train.json')
create_formatted_json('/content/dev-v1.1.json', 'squad_dev.json')

**Clone the repo**

In [ ]:
!rm -rf llama-finetuning

In [ ]:
!git clone https://github.com/seungjun-green/llama-finetuning.git

Cloning into 'llama-finetuning'...
remote: Enumerating objects: 480, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 480 (delta 94), reused 148 (delta 56), pack-reused 290 (from 1)
Receiving objects: 100% (480/480), 149.02 KiB | 12.42 MiB/s, done.
Resolving deltas: 100% (233/233), done.


In [ ]:
import sys
sys.path.append("/content/llama-finetuning")
from scripts.finetune import Finetuner

In [ ]:
from google.colab import userdata
hf_key = userdata.get('huggingface')
!huggingface-cli login --token {hf_key}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `colab` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `colab`


### LoRA

In [ ]:
lora_trainer = Finetuner('lora',
                    '/content/llama-finetuning/configs/llama-3.2-1B_finetune_squad.json',
                    train_file_path="/content/squad_train.json",
                    dev_file_path="/content/squad_dev.json")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Total parameters: 1236666368
Trainable parameters: 851968


In [ ]:
lora_trainer.train()

Epoch 1:   0%|          | 0/5475 [00:00<?, ?it/s]

Epoch 1, Step 801, Loss: 2.1248
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch0_step800_loss2.1248
Epoch 1, Step 1601, Loss: 1.9883
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch0_step1600_loss1.9883
Epoch 1, Step 2401, Loss: 1.8669
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch0_step2400_loss1.8669
Epoch 1, Step 3201, Loss: 1.8581
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch0_step3200_loss1.8581
Epoch 1, Step 4001, Loss: 1.8191
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch0_step4000_loss1.8191
Epoch 1, Step 4801, Loss: 1.8006
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch0_step4800_loss1.8006
Epoch 1, Step 5475, Loss: 1.7965
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch0_step5474_loss1.7965


Epoch 2:   0%|          | 0/5475 [00:00<?, ?it/s]

Epoch 2, Step 801, Loss: 1.7139
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch1_step800_loss1.7139
Epoch 2, Step 1601, Loss: 1.7211
Epoch 2, Step 2401, Loss: 1.7262
Epoch 2, Step 3201, Loss: 1.6629
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch1_step3200_loss1.6629
Epoch 2, Step 4001, Loss: 1.6474
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch1_step4000_loss1.6474
Epoch 2, Step 4801, Loss: 1.6553
Epoch 2, Step 5475, Loss: 1.6308
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch1_step5474_loss1.6308


Epoch 3:   0%|          | 0/5475 [00:00<?, ?it/s]

Epoch 3, Step 801, Loss: 1.6202
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch2_step800_loss1.6202
Epoch 3, Step 1601, Loss: 1.6225
Epoch 3, Step 2401, Loss: 1.6085
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch2_step2400_loss1.6085
Epoch 3, Step 3201, Loss: 1.6111
Epoch 3, Step 4001, Loss: 1.5946
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch2_step4000_loss1.5946
Epoch 3, Step 4801, Loss: 1.5796
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch2_step4800_loss1.5796
Epoch 3, Step 5475, Loss: 1.5855


Epoch 4:   0%|          | 0/5475 [00:00<?, ?it/s]

Epoch 4, Step 801, Loss: 1.5534
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch3_step800_loss1.5534
Epoch 4, Step 1601, Loss: 1.5712
Epoch 4, Step 2401, Loss: 1.5907
Epoch 4, Step 3201, Loss: 1.549
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch3_step3200_loss1.549
Epoch 4, Step 4001, Loss: 1.5323
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch3_step4000_loss1.5323
Epoch 4, Step 4801, Loss: 1.5279
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch3_step4800_loss1.5279
Epoch 4, Step 5475, Loss: 1.5245
[INFO] LoRA weights saved to ./fine_tuned_checkpoints/epoch3_step5474_loss1.5245


Epoch 5:   0%|          | 0/5475 [00:00<?, ?it/s]

Epoch 5, Step 801, Loss: 1.5666
Epoch 5, Step 1601, Loss: 1.5637
Epoch 5, Step 2401, Loss: 1.5458
Epoch 5, Step 3201, Loss: 1.5519
Epoch 5, Step 4001, Loss: 1.5577
Early stopping triggered after 5 consecutive logs without improvement.
